# 01. 순환 신경망(Recurrent Neural Network, RNN)

* RNN(Recurrent Neural Network)은 시퀀스(Sequence) 모델
* 입력과 출력을 시퀀스 단위로 처리하는 모델

* 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로만 향하는 신경망들을 피드 포워드 신경망(Feed Forward Neural Network)이라고 한다. 
*  RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있다.
* RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀(cell)이라고 한다. 이 셀은 이전의 값을 기억하려고 하는 일종의 **메모리 역할**을 수행하므로 이를 메모리 셀 또는 RNN 셀이라고 표현합니다.

은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있습니다. 앞으로는 현재 시점을 변수 t로 표현하겠습니다. 이는 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미합니다. 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값을 은닉 상태(hidden state)라고 합니다. 다시 말해 t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용합니다.

 <img src="https://wikidocs.net/images/page/22886/rnn_image3_ver2.PNG" width="400" height="150">

RNN 셀의 각 시점 별 입, 출력의 단위는 사용자가 정의하기 나름이지만 가장 보편적인 단위는 '단어 벡터'입니다.

### one-to-many 일대다
하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용

 ### many to one 다대일
* 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification), 또는 메일이 정상 메일인지 스팸 메일인지 판별하는 스팸 메일 분류(spam detection)에 사용  
 <img src="https://wikidocs.net/images/page/22886/rnn_image3.5.PNG" width="300" height="200">

### many-to-many 다대다
* 입력 문장으로 부터 대답 문장을 출력하는 챗봇과 입력 문장으로부터 번역된 문장을 출력하는 번역기, 개체명 인식이나 품사 태깅과 같은 작업이 속합니다. 위 그림은 개체명 인식을 수행할 때의 RNN 아키텍처를 보여줍니다.
 <img src="https://wikidocs.net/images/page/22886/rnn_image3.7.PNG" width="200" height="150">

 <img src="https://wikidocs.net/images/page/22886/rnn_image4_ver2.PNG" width="200" height="200">  
 
* 현재 시점 t에서의 은닉 상태값을 $h_{t}$라고 정의. 은닉층의 메모리 셀은 $h_{t}$를 계산하기 위해서 총 두 개의 가중치를 갖게 된다.
* 하나는 입력층에서 입력값을 위한 가중치 $W_{x}$이고, 하나는 이전 시점 t-1의 은닉 상태값인 $h_{t-1}$을 위한 가중치 $W_{h}$다.

* 배치 크기 : 1
* 단어 벡터의 차원(d): 4
* 은닉 상태의 크기($D_{h}$) : 4  
아래와 같은 RNN 은닉층 연산

<img src="https://wikidocs.net/images/page/22886/rnn_images4-5.PNG" width="400" height="100">  

* $h_{t}$를 계산하기 위해 주로 tanh가 사용되지만 relu로 바꿔 사용하는 시도도 있었다. 
* 각각의 가중치 $W_{x},W_{h}, W_{y}$의 값은 모든 시점에서 값을 동일하게 공유한다.
* 만약, 은닉층이 2개 이상일 경우에는 은닉층 2개의 가중치는 서로 다릅니다.

In [1]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_size)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

In [2]:
print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
Wx = np.random.random((hidden_size, input_size))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias)

In [4]:
print(np.shape(Wx)) # 은닉 상태의 크기 x 입력 차원
print(np.shape(Wh)) # 은닉 상태의 크기 x 은닉 상태의 크기
print(np.shape(b)) # 은닉 상태의 크기

(8, 4)
(8, 8)
(8,)


In [12]:
print(Wx) # 은닉 상태의 크기 x 입력 차원


[[0.80359163 0.56160681 0.6063306  0.64634882]
 [0.79455186 0.32500117 0.70122763 0.61880839]
 [0.27515329 0.62635891 0.46599091 0.98666077]
 [0.20836064 0.23932951 0.55716076 0.37987397]
 [0.53256814 0.28190973 0.35133858 0.5711183 ]
 [0.91285301 0.58354993 0.24776271 0.21093201]
 [0.89038671 0.18730982 0.54981708 0.27271388]
 [0.7313198  0.64432242 0.94634256 0.74537956]]


In [13]:
print(Wh) # 은닉 상태의 크기 x 은닉 상태의 크기


[[0.87153968 0.70625823 0.17591079 0.87587016 0.47643489 0.18071301
  0.69548311 0.28313246]
 [0.18068125 0.1256363  0.61488953 0.33444476 0.86741276 0.921452
  0.9738525  0.32347332]
 [0.87801097 0.79252197 0.97755382 0.18452777 0.1518104  0.7006277
  0.16338969 0.58490722]
 [0.58013854 0.90541792 0.78598093 0.07579466 0.10854701 0.57122772
  0.49504623 0.76387376]
 [0.02005964 0.63614626 0.17928258 0.68242144 0.96453781 0.92174583
  0.21411424 0.75795383]
 [0.07620526 0.61121662 0.23644334 0.974307   0.24164173 0.00964175
  0.10216932 0.36818838]
 [0.60318641 0.052421   0.23483273 0.6850027  0.66461949 0.30172585
  0.74723413 0.96915505]
 [0.32681868 0.72160697 0.95586582 0.0229351  0.68646478 0.50800056
  0.38843864 0.29392928]]


In [14]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.99382263 0.99167989 0.96127218 0.8047054  0.908074   0.9830374
  0.98594004 0.9839666 ]
 [0.99998374 0.99998377 0.99993545 0.99981064 0.9998908  0.99953252
  0.99998189 0.99987924]
 [0.99998895 0.99998898 0.99995029 0.99985396 0.99993324 0.99960468
  0.99998764 0.99990464]
 [0.99998924 0.99998996 0.99997161 0.99991031 0.99993318 0.99929195
  0.99998355 0.9999414 ]
 [0.99999669 0.99999717 0.99997503 0.99994081 0.99996684 0.99981502
  0.99999636 0.99998233]
 [0.99999628 0.99999569 0.99999102 0.99993703 0.99996922 0.99972018
  0.99999089 0.99997909]
 [0.99999743 0.9999971  0.99999175 0.99993991 0.99997684 0.99981898
  0.99999429 0.99998412]
 [0.99998098 0.99998456 0.99996809 0.99988454 0.99992048 0.99845034
  0.99997193 0.99988264]
 [0.99999851 0.99999856 0.99999545 0.99996741 0.99998412 0.99981744
  0.99999635 0.99999375]
 [0.99999649 0.99999551 0.99999281 0.99992901 0.99997239 0.99975008
  0.99998999 0.99997702]]

# 3. 파이토치의 nn.RNN()

In [16]:
import torch
import torch.nn as nn

input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

# (배치 크기 × 시점의 수 × 매 시점마다 들어가는 입력)
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size, hidden_size, batch_first=True)
# batch_first=True : 입력 텐서의 첫번째 차원이 배치 크기임을 알려줍니다.

outputs, _status = cell(inputs)

print(outputs.shape) # 모든 time-step의 hidden_state

print(_status.shape) # 최종 time-step의 hidden_state


torch.Size([1, 10, 8])
torch.Size([1, 1, 8])


# 4. 깊은 순환 신경망(Deep Recurrent Neural Network)

<img src="https://wikidocs.net/images/page/22886/rnn_image4.5_finalPNG.PNG" width="400" height="200">  

위의 그림은 순환 신경망에서 은닉층이 1개 더 추가되어 은닉층이 2개인 깊은(deep) 순환 신경망의 모습을 보여줍니다. 위의 코드에서 첫번째 은닉층은 다음 은닉층에 모든 시점에 대해서 은닉 상태 값을 다음 은닉층으로 보내주고 있습니다.  
깊은 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 num_layers에 값을 전달하여 층을 쌓습니다. 

In [19]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
print(outputs.shape) # 모든 time-step의 hidden_state
outputs, _status = cell(inputs)

torch.Size([1, 10, 8])


In [20]:
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)

torch.Size([2, 1, 8])


# 5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

* RNN이 과거 시점(time step)의 데이터들을 참고해서, 찾고자하는 정답을 예측하지만 실제 문제에서는 과거 시점의 데이터만 고려하는 것이 아니라 향후 시점의 데이터에 힌트가 있는 경우도 많습니다. 그래서 이전 시점의 데이터뿐만 아니라, 이후 시점의 데이터도 힌트로 활용하기 위해서 고안된 것이 양방향 RNN입니다.
* 은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아닙니다. 은닉층을 추가하면, 학습할 수 있는 양이 많아지지만 또한 반대로 훈련 데이터 또한 그만큼 많이 필요합니다.



In [22]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional = True)
outputs, _status = cell(inputs)
print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])
